In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, Flatten, Dense, Softmax, BatchNormalization, Dropout, Add
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers

import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
tf.__version__

'2.9.1'

# Data Pre-Processing

Open **dan_train.csv** file and split the games into a list.
Every row of csv: `PSL0000000001,1,B[pd],W[dp],B[qp],W[dc],B[nq],W[nc],B[qf],W[kd],B[ce],W[dg],B[dd],W[cc],B[fd],W[ed],B[ee],W[ec],B[ge],W[gc],B[di]`. 

Columns are:

    1. PSL0000000001: Game ID
    2. 1: Game Style
    3-... : Moves representing this game style
    
We cropped only the moves to game list as:

In [3]:
df = open('./CSVs/Tutorial_play_style_train.csv').read().splitlines()
games = [i.split(',',2)[-1] for i in df]
game_styles = [int(i.split(',',2)[-2]) for i in df]

Create a dictionary to convert the coordinates from characters to numbers

In [4]:
chars = 'abcdefghijklmnopqrst'
coordinates = {k:v for v,k in enumerate(chars)}
coordinates

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'e': 4,
 'f': 5,
 'g': 6,
 'h': 7,
 'i': 8,
 'j': 9,
 'k': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'q': 16,
 'r': 17,
 's': 18,
 't': 19}

We decided to build a DCNN model in this tutorial. We create data samples by using every move in every game, meaning that the target is to predict the next move by feeding the previous state of the table in every game for every move. Therefore, we can collect much more data samples from games.

For the simplicity, we used 2 dimensional feature map to represent the data as below:
 1. Occupied areas: mark them as 1 and the empty places as 0
 2. The last move in the table: mark the position of the last move as 1 and the rest as 0
 
The target is to predict the game style (1, 2 or 3) from the state of the game table. Later this will be one-hot encoded.

In [5]:
def prepare_input(moves):
    x = np.zeros((19,19,2))
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        x[row,column,0] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,1] = 1
    return x

In [6]:
# Check how many samples can be obtained
n_games = 0
for game in games:
    n_games += 1
print(f"Total Games: {n_games}")

Total Games: 10684


Since play style training has smaller dataset comparing to kyu or dan training, we can put the complete dataset to memory. Still, it is better to create a data generator.

In [7]:
x = []
for game in games:
    moves_list = game.split(',')
    x.append(prepare_input(moves_list))
x = np.array(x)
y = np.array(game_styles)-1

In [8]:
x.shape

(10684, 19, 19, 2)

In [9]:
y.shape

(10684,)

In [10]:
np.bincount(y)

array([3259, 3796, 3629], dtype=int64)

Target is one-hot encoded and loss is changed to `categorical_crossentropy`

In [11]:
y_hot = tf.one_hot(y, depth=3)

Dataset splitting: 90% Training, 10% validation

In [12]:
x_train, x_val, y_train, y_val = train_test_split(x, y_hot.numpy(), test_size=0.10)

# Training

### Simple DCNN Model:

In [13]:
def create_model():
    inputs = Input(shape=(19, 19, 2))
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(inputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=7, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=5, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=32, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Conv2D(kernel_size=3, filters=1, padding='same', activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Flatten()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(32, activation='relu')(outputs)
    outputs = BatchNormalization()(outputs)
    outputs = Dense(3, activation='softmax', )(outputs)
    model = Model(inputs, outputs)
    opt = Adam(learning_rate=0.00005)
    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [14]:
model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 19, 19, 2)]       0         
                                                                 
 conv2d (Conv2D)             (None, 19, 19, 32)        3168      
                                                                 
 batch_normalization (BatchN  (None, 19, 19, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 19, 19, 32)        50208     
                                                                 
 batch_normalization_1 (Batc  (None, 19, 19, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_2 (Conv2D)           (None, 19, 19, 32)        25632 

In [15]:
history = model.fit(
    x = x_train, 
    y = y_train,
    batch_size = 64,
    epochs = 20,
    validation_data=(x_val, y_val),
)

Epoch 1/20
151/151 [==============================] - 4s 15ms/step - loss: 1.4686 - accuracy: 0.3436 - val_loss: 1.1592 - val_accuracy: 0.3283
Epoch 2/20
151/151 [==============================] - 2s 12ms/step - loss: 1.1535 - accuracy: 0.4444 - val_loss: 1.2009 - val_accuracy: 0.3489
Epoch 3/20
151/151 [==============================] - 2s 12ms/step - loss: 0.9786 - accuracy: 0.5392 - val_loss: 1.2401 - val_accuracy: 0.3564
Epoch 4/20
151/151 [==============================] - 2s 12ms/step - loss: 0.8651 - accuracy: 0.6032 - val_loss: 1.2429 - val_accuracy: 0.3779
Epoch 5/20
151/151 [==============================] - 2s 12ms/step - loss: 0.7772 - accuracy: 0.6640 - val_loss: 1.2135 - val_accuracy: 0.4079
Epoch 6/20
151/151 [==============================] - 2s 12ms/step - loss: 0.7007 - accuracy: 0.7152 - val_loss: 1.2239 - val_accuracy: 0.4097
Epoch 7/20
151/151 [==============================] - 2s 12ms/step - loss: 0.6358 - accuracy: 0.7645 - val_loss: 1.2270 - val_accuracy: 0.4060

In [16]:
model.save('./model_playstyle.h5')

# Testing

**PublicUpload.csv** must be in the following form:
```
PSTPU0000000001_79,1
PSTPU0000000001_7,1
PSTPU0000000001_153,1
PSTPU0000000001_115,1
PSTPU0000000001_131,1
PSTPU0000000001_99,1
PSTPU0000000001_21,2
```

- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is to use **play_style_test_public.csv** to predict and save the results in required form.

In [17]:
df = open('./CSVs/Tutorial_play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

chars = 'abcdefghijklmnopqrst'
chartonumbers = {k:v for k,v in enumerate(chars)}

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)


348/348 [==============================] - 1s 3ms/step


Save results to **PublicUpload.csv**.

In [18]:
with open('./PublicUpload.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

# End of Tutorial

You are free to use more modern NN architectures, a better pre-processing, feature extraction methods to achieve much better accuracy!